
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

## Create Tables
Run the cell below to create tables for the questions in this notebook. 

In [0]:
%run ../Utilities/07-CreateTables

Declared the following table:
 finances

Declared the following table:
 charges

## Question 1: Transform
### Summary
Use the **`TRANSFORM`** function and the table **`finances`** to calculate **`interest`** for all cards issued to each user. 


### Steps to complete
Write a SQL query that achieves the following: 
* Displays cardholder's **`firstName`**, **`lastName`**, and a new column named **`interest`**
* Uses **`TRANSFORM`** to extract charges for each card in the expenses column and calculates interest owed assuming a rate of 6.25%. 
* Stores the new values as an array in the interest column
* Stores results in a temporary table named `q1Results`

A properly completed solution should return a view that looks similar to this:

| firstName | lastName | interest |
|----------- |---------|----------|
|Lance|Da Costa|[138.9, 373.55, 158.97]|

In [0]:
%sql
SELECT *
FROM finances;

id,firstName,lastName,expenses
1,Lance,Da Costa,"List(Map(lastPayment -> 2020-07-18, paymentDue -> 2020-07-18, charges -> 2222.46, currency -> USD, cardType -> jcb), Map(lastPayment -> 2020-07-01, paymentDue -> 2020-07-22, charges -> 5976.76, currency -> USD, cardType -> jcb), Map(lastPayment -> 2020-07-12, paymentDue -> 2020-07-22, charges -> 2543.55, currency -> USD, cardType -> visa))"
2,Emilie,Newlove,"List(Map(lastPayment -> 2020-07-10, paymentDue -> 2020-07-08, charges -> 6344.33, currency -> USD, cardType -> bankcard))"
3,Alvy,Records,"List(Map(lastPayment -> 2020-07-02, paymentDue -> 2020-07-29, charges -> 9170.83, currency -> USD, cardType -> mastercard), Map(lastPayment -> 2020-07-06, paymentDue -> 2020-07-22, charges -> 3201.67, currency -> USD, cardType -> maestro), Map(lastPayment -> 2020-07-28, paymentDue -> 2020-07-08, charges -> 6087.61, currency -> USD, cardType -> americanexpress), Map(lastPayment -> 2020-07-12, paymentDue -> 2020-07-21, charges -> 3392.61, currency -> USD, cardType -> americanexpress))"
4,Jena,Fairley,"List(Map(lastPayment -> 2020-07-25, paymentDue -> 2020-07-17, charges -> 9726.16, currency -> USD, cardType -> jcb), Map(lastPayment -> 2020-07-27, paymentDue -> 2020-07-14, charges -> 2578.18, currency -> USD, cardType -> americanexpress))"
5,Klarika,Pady,"List(Map(lastPayment -> 2020-07-12, paymentDue -> 2020-07-17, charges -> 8460.46, currency -> USD, cardType -> bankcard), Map(lastPayment -> 2020-07-11, paymentDue -> 2020-07-08, charges -> 8573.72, currency -> USD, cardType -> bankcard))"


In [0]:
%sql
SELECT firstName, lastName, 
TRANSFORM(expenses, card -> ROUND(card.charges * 0.0625, 2)) AS interest 
FROM finances;

firstName,lastName,interest
Lance,Da Costa,"List(138.9, 373.55, 158.97)"
Emilie,Newlove,List(396.52)
Alvy,Records,"List(573.18, 200.1, 380.48, 212.04)"
Jena,Fairley,"List(607.89, 161.14)"
Klarika,Pady,"List(528.78, 535.86)"


## Question 2: Exists
### Summary
Use the table from Question 1, **`finances`**, to flag users whose records indicate that they made a late payment. 

### Steps to complete
Write a SQL query that achieves the following: 
* Displays cardholder's **`firstName`**, **`lastName`**, and a new column named **`lateFee`**
* Uses the EXISTS function to flag customers who have made been charged a late payment fee.
* Store the results in a temporary view named **`q2Results`**
   
A properly completed solution should return a DataFrame that looks similar to this:

| firstName | lastName | lateFee |
|---------- |----------| ------- |
|Lance|DaCosta |true|

In [0]:
%sql
CREATE 
OR REPLACE TEMPORARY VIEW q2Results AS
  SELECT firstName, lastName,
  EXISTS(expenses, card -> TO_DATE(card.paymentDue) > TO_DATE(card.lastPayment)) AS lateFee
  FROM finances;

In [0]:
%sql
SELECT 
  * 
FROM 
  q2Results;

firstName,lastName,lateFee
Lance,Da Costa,true
Emilie,Newlove,false
Alvy,Records,true
Jena,Fairley,false
Klarika,Pady,true


## Question 3: Reduce
### Summary
Use the **`REDUCE`** function to produce a query on the table **`charges`** that calculates total charges in dollars and total charges in Japanese Yen. 

### Steps to complete
Write a SQL query that achieves the following: 
* Uses the **`REDUCE`** function to calculate the total charges in US Dollars (given)
* Uses the **`REDUCE`** function to convert the total charges to Japanese Yen using a conversion rate where 1 USD = 107.26 JPY 
* Stores the results in a temporary table named **`q3Results`**
   
**NOTE:** In the `REDUCE` function, the accumulator must be of the same type as the input. You will have to `CAST` the accumulator as a `DOUBLE` to use this function with this data. 
example: `CAST (0 AS DOUBLE`)

A properly completed solution should return a DataFrame that looks similar to this:

| firstName | lastName | allCharges | totalDollars | totalYen |
|---------- |----------| ------- | --------| ----------|
|Lance|DaCosta |["2222.46", "5976.76", "2543.55"]|10742.77|1152269.51|

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW q3Results AS
  SELECT firstName, lastName, allCharges, 
  REDUCE (allCharges, CAST(0 AS DOUBLE), (charge, acc) -> charge + acc) AS totalDollars,
  REDUCE (allCharges, CAST(0 AS DOUBLE), (charge, acc) -> charge + acc, acc -> ROUND(acc * 107.26, 2)) AS totalYen
  FROM charges;

In [0]:
%sql
SELECT
 * 
FROM
 q3Results;

firstName,lastName,allCharges,totalDollars,totalYen
Lance,Da Costa,"List(2222.46, 5976.76, 2543.55)",10742.77,1152269.51
Emilie,Newlove,List(6344.33),6344.33,680492.84
Alvy,Records,"List(9170.83, 3201.67, 6087.61, 3392.61)",21852.72,2343922.75
Jena,Fairley,"List(9726.16, 2578.18)",12304.34,1319763.51
Klarika,Pady,"List(8460.46, 8573.72)",17034.18,1827086.15


&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>